# IMPORT LIBRARIES

In [2]:
from tqdm import *
import sys
sys.path.append("../")

import config2 as config
import MODEL
import UTILS

import os
import time
import argparse
import math
import numpy as np
import pandas as pd
from collections import Counter
import random

import torch
from torch.utils.data.dataset import Dataset
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

torch.manual_seed(0)

# HYPERPARAMETERS

In [3]:
parser = argparse.ArgumentParser(
    description='arguments')
parser.add_argument('--init', type=int, default=0, help='init number')
parser.add_argument('--fold', type=int, default=0, help='fold number')
parser.add_argument('--few_shot_years', type=int, default=7, help='few_shot_years')
parser.add_argument('--model_name', type=str, default='tamlstm', help='model_name')
parser.add_argument('--date', type=str, default='20230420', help='date')

args = parser.parse_args()
#args, unknown = parser.parse_known_args()



In [4]:
# TIME SERIES INFO
window = config.window
stride = config.stride

# CHANNELS INFO
dynamic_channels = config.dynamic_channels
output_channels = config.output_channels

no_normalize_channels = config.no_normalize_channels
normalize_channels = config.normalize_channels

static_channels = config.dynamic_channels  # WE DONT USE THIS

# LABELS INFO
unknown = config.unknown

# MODEL INFO
model_name = args.model_name #"test_model"
forward_code_dim = config.forward_code_dim
latent_code_dim = config.latent_code_dim
device = torch.device(config.device)
dropout = config.dropout

# TRAIN INFO 
train = config.train
batch_size = config.batch_size
epochs = config.epochs
learning_rate = config.learning_rate
meta_learning_rate =config.meta_learning_rate

init = args.init  
fold = 0
num_inner_steps = 5
max_patience = 20
few_shot_years = 1# args.few_shot_years
train_percent = 0.8

print("Hyperparameters:{}".format(model_name))
print("window : {}".format(window))
print("stride : {}".format(stride))
print("dynamic_channels : {}".format(dynamic_channels))
print("output_channels : {}".format(output_channels))
print("unknown : {}".format(unknown))
print("model_name : {}".format(model_name))
print("forward_code_dim : {}".format(forward_code_dim))
print("latent_code_dim : {}".format(latent_code_dim))
print("device : {}".format(device))
print("dropout : {}".format(dropout))
print("train : {}".format(train))
print("batch_size : {}".format(batch_size))
print("epochs : {}".format(epochs))
print("learning_rate : {}".format(learning_rate))
print("meta_learning_rate : {}".format(meta_learning_rate))
print("num_inner_steps : {}".format(num_inner_steps))
print("init : {}".format(init))
print("fold : {}".format(fold))
print("max_patience : {}".format(max_patience))
print("few_shot_years : {}".format(few_shot_years))

Hyperparameters:test_model
window : 30
stride : 15
dynamic_channels : [0, 1, 2, 3, 4]
output_channels : [5]
unknown : nan
model_name : test_model
forward_code_dim : 350
latent_code_dim : 64
device : cuda
dropout : 0.4
train : True
batch_size : 64
epochs : 1
learning_rate : 0.001
meta_learning_rate : 0.001
num_inner_steps : 5
init : 0
fold : 0
max_patience : 20
few_shot_years : 1


# DEFINE DIRECTORIES

In [5]:
DATE = args.date #"20240327" #
PREPROCESSED_DIR = config.PREPROCESSED_DIR
RESULT_DIR = os.path.join(config.RESULT_DIR, DATE)
MODEL_DIR = os.path.join(config.MODEL_DIR, DATE)

# LOAD DATA

In [6]:
def load_dataset(file):
    dataset = np.load(os.path.join(PREPROCESSED_DIR, "{}.npz".format(file)), allow_pickle=True)
    return dataset

def get_data(dataset,preprocessed=True):
    data = dataset["data"]
    if preprocessed:
        data_mean = dataset["train_data_means"]
        data_std =  dataset["train_data_stds"]

        #print(data_mean.shape)
        normalized_data = np.zeros_like(data)
        if len(data.shape)==4:
            for feature in range(data_mean.shape[0]):
                if data_std[feature]!=0:
                    normalized_data[:,:,:,feature] = (data[:,:,:,feature] - data_mean[feature])/data_std[feature]
                else:
                    normalized_data[:,:,:,feature] = data[:,:,:,feature]
                normalized_data[:,:,:,no_normalize_channels] = data[:,:,:,no_normalize_channels]
        else:
            normalized_data[:,:,normalize_channels] = data[:,:,normalize_channels] - data_mean[normalize_channels]/data_std[normalize_channels]
            # normalized_data[:,:,-1] = data[:,:,-1]
        data = normalized_data
    data = np.nan_to_num(data, nan=unknown)
    #print(data.shape)
    return data


# def get_data(dataset, preprocessed=True):
#     data = dataset["data"]
#     if preprocessed:
#         data_mean = dataset["train_data_means"]
#         data_std =  dataset["train_data_stds"]

#         # Reshape data to (1, 145*72, 30, 24)
#         data = np.expand_dims(data, axis=0)
#         data = np.reshape(data, (1, -1, data.shape[2], data.shape[3]))

#         # Normalize the data
#         normalized_data = np.zeros_like(data)
#         if len(data.shape) == 4:
#             for feature in range(data_mean.shape[0]):
#                 if data_std[feature] != 0:
#                     normalized_data[:, :, :, feature] = (data[:, :, :, feature] - data_mean[feature]) / data_std[feature]
#                 else:
#                     normalized_data[:, :, :, feature] = data[:, :, :, feature]
#         else:
#             normalized_data[:, :, normalize_channels] = (data[:, :, normalize_channels] - data_mean[normalize_channels]) / data_std[normalize_channels]

#         data = normalized_data

#     # Replace NaN values with unknown
#     unknown = 0  # Define unknown here, assuming it's 0
#     data = np.nan_to_num(data, nan=unknown)
#     return data



In [7]:
file, index = "strided_train", "in_indices"
dataset = load_dataset(file)
data = get_data(dataset)

#print(data[:,:,:,-1])
data.shape

(145, 72, 30, 24)

In [8]:
file, index = "test", "in_indices"
dataset = load_dataset(file)
data = get_data(dataset)

#print(data[:,:,:,-1])
data.shape

/tmp/ipykernel_3010059/1520128585.py:21: RuntimeWarning: invalid value encountered in divide
  normalized_data[:,:,normalize_channels] = data[:,:,normalize_channels] - data_mean[normalize_channels]/data_std[normalize_channels]


(15, 1095, 24)

In [9]:
print(dataset["train_data_stds"][output_channels])
print(dataset["train_data_means"][output_channels])

[3.91537482]
[3.34140129]


# TRAIN MODEL

In [10]:
if train:

    # BUILD MODEL
    inverse_model = getattr(MODEL, "ae")(input_channels=len(dynamic_channels)+len(output_channels), code_dim=latent_code_dim, hidden_dim=latent_code_dim, output_channels=len(static_channels), device=device)
    inverse_model = inverse_model.to(device)
    pytorch_total_params = sum(p.numel() for p in inverse_model.parameters() if p.requires_grad)
    print(inverse_model)
    forward_model = getattr(MODEL, "tamlstm")(input_dynamic_channels=len(dynamic_channels), input_static_channels=latent_code_dim, hidden_dim=forward_code_dim, output_channels=len(output_channels), dropout=dropout)
    forward_model = forward_model.to(device)
    pytorch_total_params += sum(p.numel() for p in forward_model.parameters() if p.requires_grad)
    print(forward_model)
    print("#Parameters:{}".format(pytorch_total_params))
    mse_criterion = torch.nn.MSELoss(reduction="none")
    optimizer = torch.optim.Adam(list(inverse_model.parameters())+list(forward_model.parameters()), lr=meta_learning_rate)

    train_loss = []
    valid_loss = []
    test_loss = []
    min_loss = 10000
    patience = 0 

    for epoch in range(1,epochs+1):
        

        start = time.time()

        # LOSS ON TRAIN SET
        inverse_model.train()
        forward_model.train()

        # LOAD DATA
        file, index = "strided_train", "in_indices"
        dataset = load_dataset(file)
        data = get_data(dataset)
        nodes, years, window, channels = data.shape       # HERE TAKE A LOOK
        # GET RANDOM Train and val Basins  

        # random_basin = random.sample(range(nodes), nodes)        
        # basin_train = random_basin[:int(train_percent*nodes)]
        # basin_valid = random_basin[int(train_percent*nodes):]     
        data_train = data[:,:int(0.8*years),:,:]      
        # data_train = data[basin_train,:,:,:]      
        nodes, years, window, channels = data_train.shape    #HERE TAKE A LOOK

        # GET ANCHOR AND POSITIVE YEARS
        anchor_years = np.zeros((nodes, years))
        for node in range(nodes):
            anchor_years[node] = random.sample(range(years), years)
        anchor_years = anchor_years.astype(np.int64)
        positive_years = np.zeros((nodes, years))
        for node in range(nodes):
            positive_years[node] = random.sample(range(years), years)
        positive_years = positive_years.astype(np.int64)
        # print(anchor_years.shape, positive_years.shape)

    # 		# GET TRAIN AND VALIDATION SET
    # 		anchor_years = anchor_years[:,:int(train_percent*years)]
    # 		anchor_years = anchor_years[:,int(train_percent*years):]
    # 		positive_years = positive_years[:,:int(train_percent*years)]
    # 		positive_years = positive_years[:,int(train_percent*years):]
        # print(anchor_years.shape, anchor_years.shape, positive_years.shape, positive_years.shape)

        # LOSS ON TRAIN SET
        inverse_model.train()
        forward_model.train()
        epoch_loss = 0
        epoch_recon_loss = 0
        epoch_contrastive_loss = 0
        epoch_static_loss = 0
        epoch_forward_loss = 0
        for year in range(anchor_years.shape[1]):

            #Get (anchor,positive) Instances for each node
            anchor_data = data_train[np.arange(nodes), anchor_years[:, year]]
            positive_data = data_train[np.arange(nodes), positive_years[:, year]]
            # print(anchor_data.shape, positive_data.shape)

            # Remove pairs where (anchor,positive) years are same
            keep_idx = anchor_years[:, year] != positive_years[:, year]
            anchor_data = anchor_data[keep_idx]
            positive_data = positive_data[keep_idx]
            # print(anchor_data.shape, positive_data.shape)

            # Remove pairs where (anchor,positive) basins have unknown in streamflow
            keep_idx = np.zeros((anchor_data.shape[0], 2)).astype(bool)
            keep_idx[:,0] = (anchor_data[:,:,-1]!=unknown).all(axis=1)
            keep_idx[:,1] = (positive_data[:,:,-1]!=unknown).all(axis=1)
            keep_idx = keep_idx.all(axis=1)
            anchor_data = anchor_data[keep_idx]
            positive_data = positive_data[keep_idx]
            # print(anchor_data.shape, positive_data.shape)

            random_batches = random.sample(range(anchor_data.shape[0]),anchor_data.shape[0])
            for batch in range(math.ceil(anchor_data.shape[0]/batch_size)):

                optimizer.zero_grad()

                # GET BATCH DATA FOR INVERSE MODEL
                random_batch = random_batches[batch*batch_size:(batch+1)*batch_size]
                batch_anchor_data = torch.from_numpy(anchor_data[random_batch]).to(device)
                batch_positive_data = torch.from_numpy(positive_data[random_batch]).to(device)
                batch_input = torch.cat((batch_anchor_data[:,:,dynamic_channels+output_channels], batch_positive_data[:,:,dynamic_channels+output_channels]), dim=0)
                batch_static = torch.cat((batch_anchor_data[:,0,static_channels], batch_positive_data[:,0,static_channels]), axis=0)
                # print(batch_input.shape, batch_static.shape)

                # GET INVERSE OUTPUT
                batch_code_vec, _,_,_ = inverse_model(x=batch_input.float())
                # print(batch_code_vec.shape, batch_static_pred.shape, batch_input_pred.shape)

                # GET BATCH DATA FOR FORWARD MODEL
                batch_dynamic_input = batch_positive_data[:, :, dynamic_channels].to(device)
                batch_static_input = batch_code_vec[:batch_anchor_data.shape[0]].unsqueeze(1).repeat(1, window, 1)
                batch_label = batch_positive_data[:, :, output_channels].float().to(device)
                # print(batch_dynamic_input.shape, batch_static_input.shape, batch_label.shape)

                # GET FORWARD OUTPUT
                batch_label_pred = forward_model(x_dynamic=batch_dynamic_input.float().to(device), x_static=batch_static_input.float().to(device))
                # print(batch_label_pred.shape)
                
                
                # print("natch pred datatype",batch_label_pred.dtype)
                # print("batch label datatype",batch_label.dtype)
                

                # CALCULATE LOSS
                batch_forward_loss = mse_criterion(batch_label, batch_label_pred)										# FORWARD LOSS (PER CHANNEL LOSS)
                mask = (batch_label!=unknown).float()																	# FORWARD LOSS (CREATE MASK)
                batch_forward_loss = batch_forward_loss * mask															# FORWARD LOSS (MULTIPLY MASK)
                batch_forward_loss, mask = torch.sum(batch_forward_loss, dim=2), (torch.sum(mask, dim=2)>0).float()		# FORWARD LOSS (PER INSTANCE LOSS)
                batch_forward_loss = torch.sum(batch_forward_loss)/torch.sum(mask)										# FORWARD LOSS (MEAN SEQUENCE LOSS)
                batch_loss = batch_forward_loss
                # print(batch_loss.shape, batch_loss)

                # LOSS BACKPROPOGATE
                
#                 print("natch loss datatype",batch_loss.dtype)
                
                batch_loss.backward()
                optimizer.step()

                # AGGREGATE LOSS
                epoch_loss += batch_loss.item()
                epoch_forward_loss += batch_forward_loss.item()

        epoch_loss /= ((batch+1)*(year+1))
        epoch_forward_loss /= ((batch+1)*(year+1))
        print('Epoch:{}\tTrain Loss:{:.4f}'.format(epoch, epoch_loss), end="\t")
        train_loss.append(epoch_loss)

        # SCORE ON VALIDATION SET

        inverse_model.eval()
        forward_model.eval()

        # LOAD DATA
        # data_valid = data[basin_valid,:,:,:]      
        data_valid = data[:,int(0.8*years):,:,:]      
        nodes, years, window, channels = data_valid.shape      #TAKE A LOOK HERE
        # print(nodes, years, window, channels)

        # GET ANCHOR AND POSITIVE YEARS
        anchor_years = np.zeros((nodes, years))
        for node in range(nodes):
            anchor_years[node] = random.sample(range(years), years)
        anchor_years = anchor_years.astype(np.int64)
        positive_years = np.zeros((nodes, years))
        for node in range(nodes):
            positive_years[node] = random.sample(range(years), years)
        positive_years = positive_years.astype(np.int64)
        epoch_loss = 0
        epoch_recon_loss = 0
        epoch_contrastive_loss = 0
        epoch_static_loss = 0
        epoch_forward_loss = 0

        for year in range(anchor_years.shape[1]):

            #Get (anchor,positive) Instances for each node
            anchor_data = data_valid[np.arange(nodes), anchor_years[:, year]]
            positive_data = data_valid[np.arange(nodes), positive_years[:, year]]
            # print(anchor_data.shape, positive_data.shape)

            # Remove pairs where (anchor,positive) years are same
            keep_idx = anchor_years[:, year] != positive_years[:, year]
            anchor_data = anchor_data[keep_idx]
            positive_data = positive_data[keep_idx]
            # print(anchor_data.shape, positive_data.shape)

            # Remove pairs where (anchor,positive) basins have unknown in streamflow
            keep_idx = np.zeros((anchor_data.shape[0], 2)).astype(bool)
            keep_idx[:,0] = (anchor_data[:,:,-1]!=unknown).all(axis=1)
            keep_idx[:,1] = (positive_data[:,:,-1]!=unknown).all(axis=1)
            keep_idx = keep_idx.all(axis=1)
            anchor_data = anchor_data[keep_idx]
            positive_data = positive_data[keep_idx]
            # print(anchor_data.shape, positive_data.shape)

            random_batches = random.sample(range(anchor_data.shape[0]),anchor_data.shape[0])
            for batch in range(math.ceil(anchor_data.shape[0]/batch_size)):

                # GET BATCH DATA FOR INVERSE MODEL
                random_batch = random_batches[batch*batch_size:(batch+1)*batch_size]
                batch_anchor_data = torch.from_numpy(anchor_data[random_batch]).to(device)
                batch_positive_data = torch.from_numpy(positive_data[random_batch]).to(device)
                batch_input = torch.cat((batch_anchor_data[:,:,dynamic_channels+output_channels], batch_positive_data[:,:,dynamic_channels+output_channels]), dim=0)
                batch_static = torch.cat((batch_anchor_data[:,0,static_channels], batch_positive_data[:,0,static_channels]), axis=0)
                # print(batch_input.shape, batch_static.shape)

                # GET INVERSE OUTPUT
                batch_code_vec, _,_,_ = inverse_model(x=batch_input.float())
                # print(batch_code_vec.shape, batch_static_pred.shape, batch_input_pred.shape)

                # GET BATCH DATA FOR FORWARD MODEL
                batch_dynamic_input = batch_positive_data[:, :, dynamic_channels].to(device)
                batch_static_input = batch_code_vec[:batch_anchor_data.shape[0]].unsqueeze(1).repeat(1, window, 1)
                batch_label = batch_positive_data[:, :, output_channels].to(device)
                # print(batch_dynamic_input.shape, batch_static_input.shape, batch_label.shape)

                # GET FORWARD OUTPUT
                batch_label_pred = forward_model(x_dynamic=batch_dynamic_input.float().to(device), x_static=batch_static_input.float().to(device))
                # print(batch_label_pred.shape)

                # CALCULATE LOSS
                batch_forward_loss = mse_criterion(batch_label, batch_label_pred)										# FORWARD LOSS (PER CHANNEL LOSS)
                mask = (batch_label!=unknown).float()																	# FORWARD LOSS (CREATE MASK)
                batch_forward_loss = batch_forward_loss * mask															# FORWARD LOSS (MULTIPLY MASK)
                batch_forward_loss, mask = torch.sum(batch_forward_loss, dim=2), (torch.sum(mask, dim=2)>0).float()		# FORWARD LOSS (PER INSTANCE LOSS)
                batch_forward_loss = torch.sum(batch_forward_loss)/torch.sum(mask)										# FORWARD LOSS (MEAN SEQUENCE LOSS)
                batch_loss = batch_forward_loss
                # print(batch_loss.shape, batch_loss)

                # AGGREGATE LOSS
                epoch_loss += batch_loss.item()
                epoch_forward_loss += batch_forward_loss.item()

        epoch_loss /= ((batch+1)*(year+1))
        epoch_forward_loss /= ((batch+1)*(year+1))
        print('Valid Loss:{:.4f}'.format(epoch_loss), end="\t")
        valid_loss.append(epoch_loss)
        

    # 		if min_loss>epoch_loss:        
        if min_loss>epoch_forward_loss:
            min_loss = epoch_forward_loss
            torch.save(inverse_model.state_dict(), os.path.join(MODEL_DIR, "{}_inverse".format(model_name)))
            torch.save(forward_model.state_dict(), os.path.join(MODEL_DIR, "{}_forward".format(model_name)))
            
        # SCORE ON TEST SET

        inverse_model.eval()
        forward_model.eval()

        # LOAD DATA
        file, index = "strided_test", "out_indices"
        dataset = load_dataset(file)
        data = get_data(dataset)
        nodes, years, window, channels = data.shape     #TAKE A LOOK HERE 
        nodes, years, window, channels = data.shape
        # print(nodes, years, window, channels)

        # GET ANCHOR AND POSITIVE YEARS
        anchor_years = np.zeros((nodes, years))
        for node in range(nodes):
            anchor_years[node] = random.sample(range(years), years)
        anchor_years = anchor_years.astype(np.int64)
        positive_years = np.zeros((nodes, years))
        for node in range(nodes):
            positive_years[node] = random.sample(range(years), years)
        positive_years = positive_years.astype(np.int64)
        epoch_loss = 0
        epoch_recon_loss = 0
        epoch_contrastive_loss = 0
        epoch_static_loss = 0
        epoch_forward_loss = 0

        for year in range(anchor_years.shape[1]):

            #Get (anchor,positive) Instances for each node
            anchor_data = data[np.arange(nodes), anchor_years[:, year]]
            positive_data = data[np.arange(nodes), positive_years[:, year]]
            # print(anchor_data.shape, positive_data.shape)

            # Remove pairs where (anchor,positive) years are same
            keep_idx = anchor_years[:, year] != positive_years[:, year]
            anchor_data = anchor_data[keep_idx]
            positive_data = positive_data[keep_idx]
            # print(anchor_data.shape, positive_data.shape)

            # Remove pairs where (anchor,positive) basins have unknown in streamflow
            keep_idx = np.zeros((anchor_data.shape[0], 2)).astype(bool)
            keep_idx[:,0] = (anchor_data[:,:,-1]!=unknown).all(axis=1)
            keep_idx[:,1] = (positive_data[:,:,-1]!=unknown).all(axis=1)
            keep_idx = keep_idx.all(axis=1)
            anchor_data = anchor_data[keep_idx]
            positive_data = positive_data[keep_idx]
            # print(anchor_data.shape, positive_data.shape)

            random_batches = random.sample(range(anchor_data.shape[0]),anchor_data.shape[0])
            for batch in range(math.ceil(anchor_data.shape[0]/batch_size)):

                # GET BATCH DATA FOR INVERSE MODEL
                random_batch = random_batches[batch*batch_size:(batch+1)*batch_size]
                batch_anchor_data = torch.from_numpy(anchor_data[random_batch]).to(device)
                batch_positive_data = torch.from_numpy(positive_data[random_batch]).to(device)
                batch_input = torch.cat((batch_anchor_data[:,:,dynamic_channels+output_channels], batch_positive_data[:,:,dynamic_channels+output_channels]), dim=0)
                batch_static = torch.cat((batch_anchor_data[:,0,static_channels], batch_positive_data[:,0,static_channels]), axis=0)
                # print(batch_input.shape, batch_static.shape)

                # GET INVERSE OUTPUT
                batch_code_vec, _,_,_ = inverse_model(x=batch_input.float())
                # print(batch_code_vec.shape, batch_static_pred.shape, batch_input_pred.shape)

                # GET BATCH DATA FOR FORWARD MODEL
                batch_dynamic_input = batch_positive_data[:, :, dynamic_channels].to(device)
                batch_static_input = batch_code_vec[:batch_anchor_data.shape[0]].unsqueeze(1).repeat(1, window, 1)
                batch_label = batch_positive_data[:, :, output_channels].float().to(device)
                # print(batch_dynamic_input.shape, batch_static_input.shape, batch_label.shape)

                # GET FORWARD OUTPUT
                batch_label_pred = forward_model(x_dynamic=batch_dynamic_input.float().to(device), x_static=batch_static_input.float().to(device))
                # print(batch_label_pred.shape)

                # CALCULATE LOSS
                batch_forward_loss = mse_criterion(batch_label, batch_label_pred)										# FORWARD LOSS (PER CHANNEL LOSS)
                mask = (batch_label!=unknown).float()																	# FORWARD LOSS (CREATE MASK)
                batch_forward_loss = batch_forward_loss * mask															# FORWARD LOSS (MULTIPLY MASK)
                batch_forward_loss, mask = torch.sum(batch_forward_loss, dim=2), (torch.sum(mask, dim=2)>0).float()		# FORWARD LOSS (PER INSTANCE LOSS)
                batch_forward_loss = torch.sum(batch_forward_loss)/torch.sum(mask)										# FORWARD LOSS (MEAN SEQUENCE LOSS)
                batch_loss = batch_forward_loss
                # print(batch_loss.shape, batch_loss)

                # AGGREGATE LOSS
                epoch_loss += batch_loss.item()
                epoch_forward_loss += batch_forward_loss.item()

        epoch_loss /= ((batch+1)*(year+1))
        epoch_forward_loss /= ((batch+1)*(year+1))
        print('TEST Loss:{:.4f}\tMin Loss:{:.4f}'.format(epoch_loss, min_loss))
        test_loss.append(epoch_loss)

        end = time.time()
        print("Time:{:.4f}".format(end-start))            

    # PLOT LOSS
    fig = plt.figure(figsize=(10,10))
    ax1 = fig.add_subplot(111)
    ax1.set_xlabel("#Epoch", fontsize=50)

    # PLOT TRAIN LOSS
    lns1 = ax1.plot(train_loss, color='red', marker='o', linewidth=4, label="TRAIN LOSS")

    # PLOT VALIDATION SCORE
    ax2 = ax1.twinx()
    lns2 = ax2.plot(valid_loss, color='blue', marker='o', linewidth=4, label="VAL LOSS")
    
    # PLOT TEST SCORE
    ax3 = ax1.twinx()
    lns3 = ax3.plot(test_loss, color='green', marker='o', linewidth=4, label="TEST LOSS")

    # added these three lines
    lns = lns1+lns2+lns3
    labs = [l.get_label() for l in lns]
    ax1.legend(lns, labs, loc="upper right", fontsize=40, frameon=False)

    plt.tight_layout(pad=0.0,h_pad=0.0,w_pad=0.0)
    plt.savefig(os.path.join(RESULT_DIR, "{}_SCORE.pdf".format(model_name)), format = "pdf")
    plt.close()

/home/kumarv/vashi024/.conda/envs/main_a100/lib/python3.10/site-packages/torch/cuda/__init__.py:132: UserWarning: 
    Found GPU0 Tesla K40m which is of cuda capability 3.5.
    PyTorch no longer supports this GPU because it is too old.
    The minimum cuda capability supported by this library is 3.7.
    
  warnings.warn(old_gpu_warn % (d, name, major, minor, min_arch // 10, min_arch % 10))


ae(
  (instance_encoder): Sequential(
    (0): Linear(in_features=6, out_features=64, bias=True)
    (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.2)
  )
  (temporal_encoder): LSTM(64, 64, batch_first=True, bidirectional=True)
  (code_linear): Linear(in_features=64, out_features=64, bias=True)
  (decode_linear): Linear(in_features=64, out_features=64, bias=True)
  (temporal_decoder): LSTM(64, 64, batch_first=True)
  (instance_decoder): Linear(in_features=64, out_features=6, bias=True)
  (static_out): Linear(in_features=64, out_features=5, bias=True)
)
tamlstm(
  (encoder): TAMLSTM(
    (_embeddings): ModuleList(
      (0): Linear(in_features=64, out_features=700, bias=True)
      (1): Linear(in_features=64, out_features=700, bias=True)
      (2): Linear(in_features=64, out_features=700, bias=True)
      (3): Linear(in_features=64, out_features=700, bias=True)
    )
  )
  (out): Linear(in_features=350, out_featur

# TEST MODEL

## Out of DISTRIBUTION

In [24]:



all_few_shot_years = [1,3,6,12,24]




num_inner_steps = 0

for few_shot_years in all_few_shot_years:
    start = time.time()


    # num_years = 9



    #     file, index = "strided_test", "in_indices"
    #     dataset = load_dataset(file)
    #     data_test = get_data(dataset, index,fold=fold)
    #     nodes, years, window, channels = data_test.shape
    file, index = "strided_test" ,"out_indices"
    dataset = load_dataset(file)
    data = get_data(dataset)  #[15,72,30,6]
#     data = data.reshape((1, years * nodes, window, len(output_channels)))
    data_test = data[:,48:]
#     data = data.reshape((1, years * nodes, window, len(output_channels)))

    nodes, years, window, channels = data_test.shape     #TAKE A LOOK HERE
    dataset_true = unknown*np.ones((nodes, years, window, len(output_channels)), dtype=np.float32)
#     dataset_true = unknown*np.ones((1, years * nodes, window, len(output_channels)), dtype=np.float32)
    dataset_pred = unknown*np.ones((nodes, years, window, len(output_channels)), dtype=np.float32)
#     dataset_pred = unknown*np.ones((1, years * nodes, window, len(output_channels)), dtype=np.float32)

    train_data = data[:,-((few_shot_years*2)-1):]  #HERE
    print("train data shape ", train_data.shape)
    nodes, years, window, channels = train_data.shape  
    train_data = train_data.reshape((1, years * nodes, window, -1))
    
    nodes, years, window, channels = train_data.shape    #TAKE A LOOK HERE
#     print(train_data.shape)

    # train_data = train_data[:,:(num_years*2-1),:,:]
    # train_data = train_data[:,years-num_years*2+1:,:,:]
    nodes, years, window, channels = train_data.shape     #TAKE A LOOK HERE
#     print(train_data.shape)
    
    file, index = "strided_test", "out_indices"


    # BUILD MODEL
    inverse_model = getattr(MODEL, "ae")(input_channels=len(dynamic_channels)+len(output_channels), code_dim=latent_code_dim, hidden_dim=latent_code_dim, output_channels=len(static_channels), device=device)
    inverse_model = inverse_model.to(device)
    pytorch_total_params = sum(p.numel() for p in inverse_model.parameters() if p.requires_grad)
    print(inverse_model)
    forward_model = getattr(MODEL, "tamlstm")(input_dynamic_channels=len(dynamic_channels), input_static_channels=latent_code_dim, hidden_dim=forward_code_dim, output_channels=len(output_channels), dropout=dropout)
    forward_model = forward_model.to(device)
    pytorch_total_params += sum(p.numel() for p in forward_model.parameters() if p.requires_grad)
    print(forward_model)
    print("#Parameters:{}".format(pytorch_total_params))
    criterion = torch.nn.MSELoss(reduction="none")
    test_criterion = torch.nn.MSELoss(reduction="mean")
    optimizer_embedding = torch.optim.Adam(list(inverse_model.parameters()), lr=learning_rate)
    optimizer_forward = torch.optim.Adam(list(forward_model.parameters()), lr=learning_rate)
    # print("#Parameters:{}".format(pytorch_total_params))
    # print(model)

    # LOAD MODEL
    inverse_model.load_state_dict(torch.load(os.path.join(MODEL_DIR, "{}_inverse".format(model_name))))
    inverse_model.eval()
    forward_model.load_state_dict(torch.load(os.path.join(MODEL_DIR, "{}_forward".format(model_name))))
    dataset_code = unknown*np.ones((nodes, years, latent_code_dim), dtype=np.float32)


    for year in range(years):

        #Get instances for each node
        node_data = train_data[np.arange(nodes), year]
        # print(node_data.shape)

        # Remove pairs where node have unknown in streamflow
        keep_idx = np.zeros((node_data.shape[0], 1)).astype(bool)
        keep_idx[:,0] = (node_data[:,:,-1]!=unknown).all(axis=1)
        keep_idx = keep_idx.all(axis=1)
        node_data = node_data[keep_idx]
        # print(node_data.shape)

        for batch in range(math.ceil(node_data.shape[0]/batch_size)):

            # GET BATCH DATA FOR INVERSE MODEL
            batch_data = torch.from_numpy(node_data[batch*batch_size:(batch+1)*batch_size]).to(device)
            batch_input = batch_data[:,:,dynamic_channels+output_channels]
            batch_static = batch_data[:,0,static_channels]
            # print(batch_input.shape, batch_static.shape)

            # GET INVERSE OUTPUT
            batch_code_vec, batch_static_pred, batch_input_pred,_ = inverse_model(x=batch_input.float())
            # print(batch_code_vec.shape, batch_static_pred.shape, batch_input_pred.shape)

            # STORE OUTPUT
            dataset_code[batch*batch_size:(batch+1)*batch_size, year] = batch_code_vec.detach().cpu().numpy()
    dataset_code = np.mean(dataset_code, axis=1)
    print("dataset_code",dataset_code.shape)
    total_loss = 0
    #for basin in range():
    for basin in range(nodes):
        # print(basin)
        forward_model.train()

        # Adapt
        for epoch in range(num_inner_steps):
            # print("I AM HERE epoch",epoch)
            # LOSS ON TRAIN SET
            forward_model.load_state_dict(torch.load(os.path.join(MODEL_DIR, "{}_forward".format(model_name))))
            # LOAD DATA
            # print(nodes, years, window, channels)

            for year in range(years):

                #Get instance for each node
    #             print("node DATA",node_data.shape)
                node_data = train_data[basin, year]
                # print(node_data.shape)


                optimizer_forward.zero_grad()

                # GET BATCH DATA AND LABEL
    #             print("node_data",node_data.shape)
                batch_data = torch.from_numpy(node_data).unsqueeze(0).to(device)
                batch_dynamic_input = batch_data[:, :, dynamic_channels].to(device)
                batch_static_input = batch_data[:, :, static_channels].to(device)
                batch_label = batch_data[:, :, output_channels].float().to(device)
                # print(batch_input.shape, batch_label.shape)

                # GET INVERSE OUTPUT
    #             print("batch_code_vec",dataset_code[basin].shape)
                batch_static_input = torch.from_numpy(dataset_code[basin]).to(device).unsqueeze(0).unsqueeze(1).repeat(1, window, 1)

                # GET FORWARD OUTPUT
    #             print("batch_code_vec_repeat",batch_static_input.shape)
    #             print("batch_dynamic_input",batch_dynamic_input.shape)
                batch_label_pred = forward_model(x_dynamic=batch_dynamic_input.float().to(device), x_static=batch_static_input.float().to(device))
                # print(batch_pred.shape)

                # CALCULATE LOSS
                batch_loss = criterion(batch_label, batch_label_pred)											# PER CHANNEL LOSS
                mask = (batch_label!=unknown).float()													# CREATE MASK
                batch_loss = batch_loss * mask															# MULTIPLY MASK
                batch_loss, mask = torch.sum(batch_loss, dim=2), (torch.sum(mask, dim=2)>0).float()		# PER INSTANCE LOSS
                batch_loss = torch.sum(batch_loss)/torch.sum(mask)										# MEAN SEQUENCE LOSS
                # print(batch_loss.shape)

                # LOSS BACKPROPOGATE
                batch_loss.backward()
                optimizer_forward.step()
    
        "TESTING"
        #YOUR PRED SHOULD BE COMBINED BUT THE LABEL SHOULD BE BROKEN
        
    print("before1 ", dataset_code.shape)
    dataset_code_repeat = np.repeat(dataset_code, data_test.shape[0], axis=0)
    print("after1 ", dataset_code_repeat.shape)
    for basin in range(data_test.shape[0]):
        inverse_model.eval()
        forward_model.eval()
    #     file, index = "strided_test", "test_index"
    #     dataset = load_dataset(file)
    #     data_test = get_data(dataset, index)
        _,years_test,_,_ = data_test.shape
        # for year in range(years_test):

        #Get instance for each node
        node_data_test = data_test[basin, :]
        #print(node_data_test.shape)

        # GET BATCH DATA AND LABEL
        batch_data = torch.from_numpy(node_data_test).unsqueeze(0).to(device)
        batch_dynamic_input = batch_data[:, :,:,dynamic_channels].to(device)
        batch_static_input = batch_data[:, :,:,static_channels].to(device)
        batch_label = batch_data[:, :,:,output_channels].to(device)
        # print(batch_input.shape, batch_label.shape)

        # GET INVERSE OUTPUT
#         print("batch_code_vec",dataset_code[basin].shape
        batch_static_input = torch.from_numpy(dataset_code_repeat[basin]).to(device).unsqueeze(0).unsqueeze(1).repeat(1,1, window, 1)
        batch_static_input = batch_static_input.repeat(1,years_test, 1, 1).squeeze()

        # GET OUTPUT
        
        # print(batch_dynamic_input.squeeze().shape)
        # print(batch_static_input.shape)
        
        batch_pred = forward_model(x_dynamic=batch_dynamic_input.squeeze().float(), x_static=batch_static_input.float())
        
        # print(batch_pred.shape)
        # print(batch_label.shape)

        # print(test_criterion(batch_label,batch_pred).item())
        total_loss+= test_criterion(batch_label,batch_pred).item()


        # STORE OUTPUT
#             print(dataset_true.shape)
#         print("size of batchlabel", batch_label.shape)
#         print("size of batchpred", batch_pred.shape)
#         print("size of dataset true", dataset_true.shape)
#         print("size of dataset pred", dataset_pred.shape)
        
        dataset_true[basin, :,:,:] = batch_label.detach().cpu().numpy()
        dataset_pred[basin, :,:,:] = batch_pred.detach().cpu().numpy()

    # dataset_true = (dataset_true*dataset["train_data_stds"][output_channels])+dataset["train_data_means"][output_channels]
    # dataset_pred = (dataset_pred*dataset["train_data_stds"][output_channels])+dataset["train_data_means"][output_channels]
#     print("std")
#     print(dataset["train_data_stds"])
    print(dataset["train_data_stds"][output_channels])
    
    total_loss/=(nodes)
    print(few_shot_years, "MONTHS" , "RMSE", np.sqrt(total_loss))
    
    np.save(os.path.join(RESULT_DIR, "{}_{}_{}".format(file, index, "true_{}".format(fold))), dataset_true)
    np.save(os.path.join(RESULT_DIR, "{}_{}_{}_{}".format(file, index, few_shot_years, model_name)), dataset_pred)


    end = time.time()
    print("Time:{:.4f}".format(end-start))
#         inverse_model.eval()
#         forward_model.eval()
#         _,years_test,_,_ = data_test.shape
#         # for year in range(years_test):

#         #Get instance for each node
#         node_data_test = data_test[basin, :]
# #         print("before ", dataset_code.shape)
# # #         dataset_code = np.repeat(dataset_code, node_data_test.shape, axis=0)
# #         print("after ", dataset_code.shape)
# #         print(node_data_test.shape)

#         # GET BATCH DATA AND LABEL
#         batch_data = torch.from_numpy(node_data_test).unsqueeze(0).to(device)
#         batch_label = batch_data[:, :,:,output_channels].to(device)
        
        
#         batch_data = torch.from_numpy(node_data_test).unsqueeze(0).to(device)
#         batch_dynamic_input = batch_data[:, :,:,dynamic_channels].to(device)
# #         batch_static_input = batch_data[:, :,:,static_channels].to(device)
# #         batch_label = batch_data[:, :,:,output_channels].to(device)
#         # print(batch_input.shape, batch_label.shape)

#         # GET INVERSE OUTPUT
# #         print("batch_code_vec",dataset_code[basin].shape
#         batch_static_input = torch.from_numpy(dataset_code_repeat[basin]).to(device).unsqueeze(0).unsqueeze(1).repeat(1,1, window, 1)
#         batch_static_input = batch_static_input.repeat(1,years_test, 1, 1).squeeze()

#         # GET OUTPUT
#         batch_pred = forward_model(x_dynamic=batch_dynamic_input.squeeze().float(), x_static=batch_static_input.float())
#         data_test_original = data[:,48:]
#         data_shape = data_test_original.shape
#         original_years = data_shape[1]
#         original_channels = data_shape[2]
        
#         #HERE I AM CHANGING THE DATA TEST
#         data_test = np.reshape(data_test_original, (1, data_shape[0]*data_shape[1], data_shape[2], data_shape[3]))
        
#         _,years_test,_,_ = data_test.shape
# #         print(data_test.shape)
#         node_data_test = data_test[0, :]
        
#         batch_data = torch.from_numpy(node_data_test).unsqueeze(0).to(device)
#         batch_dynamic_input = batch_data[:, :,:,dynamic_channels].to(device)
# #         batch_static_input = batch_data[:, :,:,static_channels].to(device)
#         batch_label = batch_data[:, :,:,output_channels].to(device)
#         # print(batch_input.shape, batch_label.shape)

#         # GET INVERSE OUTPUT
# #         print("batch_code_vec",dataset_code[basin].shape
#         batch_static_input = torch.from_numpy(dataset_code_repeat[basin]).to(device).unsqueeze(0).unsqueeze(1).repeat(1,1, window, 1)
#         batch_static_input = batch_static_input.repeat(1,years_test, 1, 1).squeeze()

#         # GET OUTPUT
        
#         batch_pred = forward_model(x_dynamic=batch_dynamic_input.squeeze().float(), x_static=batch_static_input.float())
# #         batch_pred_shape = batch_pred.shape
# #         print(batch_pred.shape)
# #         batch_pred = batch_pred.reshape((original_years, original_channels, 1))
        
#         # print(test_criterion(batch_label,batch_pred).item())
#         total_loss+= test_criterion(batch_label,batch_pred).item()


#         # STORE OUTPUT
#         dataset_true[0, :,:,:] = batch_label.detach().cpu().numpy()
#         dataset_pred[0, :,:,:] = batch_pred.detach().cpu().numpy()

#     # dataset_true = (dataset_true*dataset["train_data_stds"][output_channels])+dataset["train_data_means"][output_channels]
#     # dataset_pred = (dataset_pred*dataset["train_data_stds"][output_channels])+dataset["train_data_means"][output_channels]
# #     print(dataset["train_data_stds"])
#     print(dataset["train_data_stds"][output_channels])
    
#     total_loss/=(nodes)
#     print(few_shot_years, "MONTHS" , "RMSE", np.sqrt(total_loss))
    
#     np.save(os.path.join(RESULT_DIR, "{}_{}_{}".format(file, index, "true_{}".format(fold))), dataset_true)
#     np.save(os.path.join(RESULT_DIR, "{}_{}_{}_{}".format(file, index, few_shot_years, model_name)), dataset_pred)


#     end = time.time()
#     print("Time:{:.4f}".format(end-start))

train data shape  (15, 1, 30, 24)
ae(
  (instance_encoder): Sequential(
    (0): Linear(in_features=6, out_features=64, bias=True)
    (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.2)
  )
  (temporal_encoder): LSTM(64, 64, batch_first=True, bidirectional=True)
  (code_linear): Linear(in_features=64, out_features=64, bias=True)
  (decode_linear): Linear(in_features=64, out_features=64, bias=True)
  (temporal_decoder): LSTM(64, 64, batch_first=True)
  (instance_decoder): Linear(in_features=64, out_features=6, bias=True)
  (static_out): Linear(in_features=64, out_features=5, bias=True)
)
tamlstm(
  (encoder): TAMLSTM(
    (_embeddings): ModuleList(
      (0): Linear(in_features=64, out_features=700, bias=True)
      (1): Linear(in_features=64, out_features=700, bias=True)
      (2): Linear(in_features=64, out_features=700, bias=True)
      (3): Linear(in_features=64, out_features=700, bias=True)
    )
  )
  (out): 

In [ ]:
test_loss = criterion(torch.tensor(dataset_true),torch.tensor(dataset_pred))

print("pre adapt")

print(test_loss.shape)
print(torch.sqrt((torch.mean(test_loss, dim=(1,2,3)))))


print("mean",torch.mean(torch.sqrt((torch.mean(test_loss, dim=(1,2,3))))))
print("median",torch.median(torch.sqrt((torch.mean(test_loss, dim=(1,2,3))))))

In [ ]:



all_few_shot_years = [1,3,6,12,24]




num_inner_steps = 5

for few_shot_years in all_few_shot_years:
    start = time.time()


    # num_years = 9



    #     file, index = "strided_test", "in_indices"
    #     dataset = load_dataset(file)
    #     data_test = get_data(dataset, index,fold=fold)
    #     nodes, years, window, channels = data_test.shape
    file, index = "strided_test" ,"out_indices"
    dataset = load_dataset(file)
    data = get_data(dataset)  #[15,72,30,6]
#     data = data.reshape((1, years * nodes, window, len(output_channels)))
    data_test = data[:,48:]
#     data = data.reshape((1, years * nodes, window, len(output_channels)))

    nodes, years, window, channels = data_test.shape     #TAKE A LOOK HERE
    dataset_true = unknown*np.ones((nodes, years, window, len(output_channels)), dtype=np.float32)
#     dataset_true = unknown*np.ones((1, years * nodes, window, len(output_channels)), dtype=np.float32)
    dataset_pred = unknown*np.ones((nodes, years, window, len(output_channels)), dtype=np.float32)
#     dataset_pred = unknown*np.ones((1, years * nodes, window, len(output_channels)), dtype=np.float32)

    train_data = data[:,-((few_shot_years*2)-1):]  #HERE
    print("train data shape ", train_data.shape)
    nodes, years, window, channels = train_data.shape  
    train_data = train_data.reshape((1, years * nodes, window, -1))
    
    nodes, years, window, channels = train_data.shape    #TAKE A LOOK HERE
#     print(train_data.shape)

    # train_data = train_data[:,:(num_years*2-1),:,:]
    # train_data = train_data[:,years-num_years*2+1:,:,:]
    nodes, years, window, channels = train_data.shape     #TAKE A LOOK HERE
#     print(train_data.shape)
    
    file, index = "strided_test", "out_indices"


    # BUILD MODEL
    inverse_model = getattr(MODEL, "ae")(input_channels=len(dynamic_channels)+len(output_channels), code_dim=latent_code_dim, hidden_dim=latent_code_dim, output_channels=len(static_channels), device=device)
    inverse_model = inverse_model.to(device)
    pytorch_total_params = sum(p.numel() for p in inverse_model.parameters() if p.requires_grad)
    print(inverse_model)
    forward_model = getattr(MODEL, "tamlstm")(input_dynamic_channels=len(dynamic_channels), input_static_channels=latent_code_dim, hidden_dim=forward_code_dim, output_channels=len(output_channels), dropout=dropout)
    forward_model = forward_model.to(device)
    pytorch_total_params += sum(p.numel() for p in forward_model.parameters() if p.requires_grad)
    print(forward_model)
    print("#Parameters:{}".format(pytorch_total_params))
    criterion = torch.nn.MSELoss(reduction="none")
    test_criterion = torch.nn.MSELoss(reduction="mean")
    optimizer_embedding = torch.optim.Adam(list(inverse_model.parameters()), lr=learning_rate)
    optimizer_forward = torch.optim.Adam(list(forward_model.parameters()), lr=learning_rate)
    # print("#Parameters:{}".format(pytorch_total_params))
    # print(model)

    # LOAD MODEL
    inverse_model.load_state_dict(torch.load(os.path.join(MODEL_DIR, "{}_inverse".format(model_name))))
    inverse_model.eval()
    forward_model.load_state_dict(torch.load(os.path.join(MODEL_DIR, "{}_forward".format(model_name))))
    dataset_code = unknown*np.ones((nodes, years, latent_code_dim), dtype=np.float32)


    for year in range(years):

        #Get instances for each node
        node_data = train_data[np.arange(nodes), year]
        # print(node_data.shape)

        # Remove pairs where node have unknown in streamflow
        keep_idx = np.zeros((node_data.shape[0], 1)).astype(bool)
        keep_idx[:,0] = (node_data[:,:,-1]!=unknown).all(axis=1)
        keep_idx = keep_idx.all(axis=1)
        node_data = node_data[keep_idx]
        # print(node_data.shape)

        for batch in range(math.ceil(node_data.shape[0]/batch_size)):

            # GET BATCH DATA FOR INVERSE MODEL
            batch_data = torch.from_numpy(node_data[batch*batch_size:(batch+1)*batch_size]).to(device)
            batch_input = batch_data[:,:,dynamic_channels+output_channels]
            batch_static = batch_data[:,0,static_channels]
            # print(batch_input.shape, batch_static.shape)

            # GET INVERSE OUTPUT
            batch_code_vec, batch_static_pred, batch_input_pred,_ = inverse_model(x=batch_input.float())
            # print(batch_code_vec.shape, batch_static_pred.shape, batch_input_pred.shape)

            # STORE OUTPUT
            dataset_code[batch*batch_size:(batch+1)*batch_size, year] = batch_code_vec.detach().cpu().numpy()
    dataset_code = np.mean(dataset_code, axis=1)
    print("dataset_code",dataset_code.shape)
    total_loss = 0
    #for basin in range():
    for basin in range(nodes):
        # print(basin)
        forward_model.train()

        # Adapt
        for epoch in range(num_inner_steps):
            # print("I AM HERE epoch",epoch)
            # LOSS ON TRAIN SET
            forward_model.load_state_dict(torch.load(os.path.join(MODEL_DIR, "{}_forward".format(model_name))))
            # LOAD DATA
            # print(nodes, years, window, channels)

            for year in range(years):

                #Get instance for each node
    #             print("node DATA",node_data.shape)
                node_data = train_data[basin, year]
                # print(node_data.shape)


                optimizer_forward.zero_grad()

                # GET BATCH DATA AND LABEL
    #             print("node_data",node_data.shape)
                batch_data = torch.from_numpy(node_data).unsqueeze(0).to(device)
                batch_dynamic_input = batch_data[:, :, dynamic_channels].to(device)
                batch_static_input = batch_data[:, :, static_channels].to(device)
                batch_label = batch_data[:, :, output_channels].float().to(device)
                # print(batch_input.shape, batch_label.shape)

                # GET INVERSE OUTPUT
    #             print("batch_code_vec",dataset_code[basin].shape)
                batch_static_input = torch.from_numpy(dataset_code[basin]).to(device).unsqueeze(0).unsqueeze(1).repeat(1, window, 1)

                # GET FORWARD OUTPUT
    #             print("batch_code_vec_repeat",batch_static_input.shape)
    #             print("batch_dynamic_input",batch_dynamic_input.shape)
                batch_label_pred = forward_model(x_dynamic=batch_dynamic_input.float().to(device), x_static=batch_static_input.float().to(device))
                # print(batch_pred.shape)

                # CALCULATE LOSS
                batch_loss = criterion(batch_label, batch_label_pred)											# PER CHANNEL LOSS
                mask = (batch_label!=unknown).float()													# CREATE MASK
                batch_loss = batch_loss * mask															# MULTIPLY MASK
                batch_loss, mask = torch.sum(batch_loss, dim=2), (torch.sum(mask, dim=2)>0).float()		# PER INSTANCE LOSS
                batch_loss = torch.sum(batch_loss)/torch.sum(mask)										# MEAN SEQUENCE LOSS
                # print(batch_loss.shape)

                # LOSS BACKPROPOGATE
                batch_loss.backward()
                optimizer_forward.step()
    
        "TESTING"
        #YOUR PRED SHOULD BE COMBINED BUT THE LABEL SHOULD BE BROKEN
        
    print("before1 ", dataset_code.shape)
    dataset_code_repeat = np.repeat(dataset_code, data_test.shape[0], axis=0)
    print("after1 ", dataset_code_repeat.shape)
    for basin in range(data_test.shape[0]):
        inverse_model.eval()
        forward_model.eval()
    #     file, index = "strided_test", "test_index"
    #     dataset = load_dataset(file)
    #     data_test = get_data(dataset, index)
        _,years_test,_,_ = data_test.shape
        # for year in range(years_test):

        #Get instance for each node
        node_data_test = data_test[basin, :]
        #print(node_data_test.shape)

        # GET BATCH DATA AND LABEL
        batch_data = torch.from_numpy(node_data_test).unsqueeze(0).to(device)
        batch_dynamic_input = batch_data[:, :,:,dynamic_channels].to(device)
        batch_static_input = batch_data[:, :,:,static_channels].to(device)
        batch_label = batch_data[:, :,:,output_channels].to(device)
        # print(batch_input.shape, batch_label.shape)

        # GET INVERSE OUTPUT
#         print("batch_code_vec",dataset_code[basin].shape
        batch_static_input = torch.from_numpy(dataset_code_repeat[basin]).to(device).unsqueeze(0).unsqueeze(1).repeat(1,1, window, 1)
        batch_static_input = batch_static_input.repeat(1,years_test, 1, 1).squeeze()

        # GET OUTPUT
        
        # print(batch_dynamic_input.squeeze().shape)
        # print(batch_static_input.shape)
        
        batch_pred = forward_model(x_dynamic=batch_dynamic_input.squeeze().float(), x_static=batch_static_input.float())
        
        # print(batch_pred.shape)
        # print(batch_label.shape)

        # print(test_criterion(batch_label,batch_pred).item())
        total_loss+= test_criterion(batch_label,batch_pred).item()


        # STORE OUTPUT
#             print(dataset_true.shape)
#         print("size of batchlabel", batch_label.shape)
#         print("size of batchpred", batch_pred.shape)
#         print("size of dataset true", dataset_true.shape)
#         print("size of dataset pred", dataset_pred.shape)
        
        dataset_true[basin, :,:,:] = batch_label.detach().cpu().numpy()
        dataset_pred[basin, :,:,:] = batch_pred.detach().cpu().numpy()

    # dataset_true = (dataset_true*dataset["train_data_stds"][output_channels])+dataset["train_data_means"][output_channels]
    # dataset_pred = (dataset_pred*dataset["train_data_stds"][output_channels])+dataset["train_data_means"][output_channels]
#     print("std")
#     print(dataset["train_data_stds"])
    print(dataset["train_data_stds"][output_channels])
    
    total_loss/=(nodes)
    print(few_shot_years, "MONTHS" , "RMSE", np.sqrt(total_loss))
    
    np.save(os.path.join(RESULT_DIR, "{}_{}_{}".format(file, index, "true_{}".format(fold))), dataset_true)
    np.save(os.path.join(RESULT_DIR, "{}_{}_{}_{}".format(file, index, few_shot_years, model_name)), dataset_pred)


    end = time.time()
    print("Time:{:.4f}".format(end-start))
#         inverse_model.eval()
#         forward_model.eval()
#         _,years_test,_,_ = data_test.shape
#         # for year in range(years_test):

#         #Get instance for each node
#         node_data_test = data_test[basin, :]
# #         print("before ", dataset_code.shape)
# # #         dataset_code = np.repeat(dataset_code, node_data_test.shape, axis=0)
# #         print("after ", dataset_code.shape)
# #         print(node_data_test.shape)

#         # GET BATCH DATA AND LABEL
#         batch_data = torch.from_numpy(node_data_test).unsqueeze(0).to(device)
#         batch_label = batch_data[:, :,:,output_channels].to(device)
        
        
#         batch_data = torch.from_numpy(node_data_test).unsqueeze(0).to(device)
#         batch_dynamic_input = batch_data[:, :,:,dynamic_channels].to(device)
# #         batch_static_input = batch_data[:, :,:,static_channels].to(device)
# #         batch_label = batch_data[:, :,:,output_channels].to(device)
#         # print(batch_input.shape, batch_label.shape)

#         # GET INVERSE OUTPUT
# #         print("batch_code_vec",dataset_code[basin].shape
#         batch_static_input = torch.from_numpy(dataset_code_repeat[basin]).to(device).unsqueeze(0).unsqueeze(1).repeat(1,1, window, 1)
#         batch_static_input = batch_static_input.repeat(1,years_test, 1, 1).squeeze()

#         # GET OUTPUT
#         batch_pred = forward_model(x_dynamic=batch_dynamic_input.squeeze().float(), x_static=batch_static_input.float())
#         data_test_original = data[:,48:]
#         data_shape = data_test_original.shape
#         original_years = data_shape[1]
#         original_channels = data_shape[2]
        
#         #HERE I AM CHANGING THE DATA TEST
#         data_test = np.reshape(data_test_original, (1, data_shape[0]*data_shape[1], data_shape[2], data_shape[3]))
        
#         _,years_test,_,_ = data_test.shape
# #         print(data_test.shape)
#         node_data_test = data_test[0, :]
        
#         batch_data = torch.from_numpy(node_data_test).unsqueeze(0).to(device)
#         batch_dynamic_input = batch_data[:, :,:,dynamic_channels].to(device)
# #         batch_static_input = batch_data[:, :,:,static_channels].to(device)
#         batch_label = batch_data[:, :,:,output_channels].to(device)
#         # print(batch_input.shape, batch_label.shape)

#         # GET INVERSE OUTPUT
# #         print("batch_code_vec",dataset_code[basin].shape
#         batch_static_input = torch.from_numpy(dataset_code_repeat[basin]).to(device).unsqueeze(0).unsqueeze(1).repeat(1,1, window, 1)
#         batch_static_input = batch_static_input.repeat(1,years_test, 1, 1).squeeze()

#         # GET OUTPUT
        
#         batch_pred = forward_model(x_dynamic=batch_dynamic_input.squeeze().float(), x_static=batch_static_input.float())
# #         batch_pred_shape = batch_pred.shape
# #         print(batch_pred.shape)
# #         batch_pred = batch_pred.reshape((original_years, original_channels, 1))
        
#         # print(test_criterion(batch_label,batch_pred).item())
#         total_loss+= test_criterion(batch_label,batch_pred).item()


#         # STORE OUTPUT
#         dataset_true[0, :,:,:] = batch_label.detach().cpu().numpy()
#         dataset_pred[0, :,:,:] = batch_pred.detach().cpu().numpy()

#     # dataset_true = (dataset_true*dataset["train_data_stds"][output_channels])+dataset["train_data_means"][output_channels]
#     # dataset_pred = (dataset_pred*dataset["train_data_stds"][output_channels])+dataset["train_data_means"][output_channels]
# #     print(dataset["train_data_stds"])
#     print(dataset["train_data_stds"][output_channels])
    
#     total_loss/=(nodes)
#     print(few_shot_years, "MONTHS" , "RMSE", np.sqrt(total_loss))
    
#     np.save(os.path.join(RESULT_DIR, "{}_{}_{}".format(file, index, "true_{}".format(fold))), dataset_true)
#     np.save(os.path.join(RESULT_DIR, "{}_{}_{}_{}".format(file, index, few_shot_years, model_name)), dataset_pred)


#     end = time.time()
#     print("Time:{:.4f}".format(end-start))

In [ ]:
print(torch.sqrt(test_criterion(torch.tensor(dataset_true),torch.tensor(dataset_pred))))

In [ ]:
test_loss = criterion(torch.tensor(dataset_true),torch.tensor(dataset_pred))


print(test_loss.shape)
print(torch.sqrt((torch.mean(test_loss, dim=(1,2,3)))))


print("mean",torch.mean(torch.sqrt((torch.mean(test_loss, dim=(1,2,3))))))
print("median",torch.median(torch.sqrt((torch.mean(test_loss, dim=(1,2,3))))))

In [25]:



all_few_shot_years = [1,3,6,12,24]




num_inner_steps = 0

for few_shot_years in all_few_shot_years:
    start = time.time()


    # num_years = 9



    #     file, index = "strided_test", "in_indices"
    #     dataset = load_dataset(file)
    #     data_test = get_data(dataset, index,fold=fold)
    #     nodes, years, window, channels = data_test.shape
    file, index = "strided_test" ,"out_indices"
    dataset = load_dataset(file)
    data = get_data(dataset)  #[15,72,30,6]
#     data = data.reshape((1, years * nodes, window, len(output_channels)))
    data_test = data[:,48:]
#     data = data.reshape((1, years * nodes, window, len(output_channels)))

    nodes, years, window, channels = data_test.shape     #TAKE A LOOK HERE
    dataset_true = unknown*np.ones((nodes, years, window, len(output_channels)), dtype=np.float32)
#     dataset_true = unknown*np.ones((1, years * nodes, window, len(output_channels)), dtype=np.float32)
    dataset_pred = unknown*np.ones((nodes, years, window, len(output_channels)), dtype=np.float32)
#     dataset_pred = unknown*np.ones((1, years * nodes, window, len(output_channels)), dtype=np.float32)

    train_data = data[:,-((few_shot_years*2)-1):]  #HERE
    print("train data shape ", train_data.shape)
    nodes, years, window, channels = train_data.shape  
    train_data = train_data.reshape((1, years * nodes, window, -1))
    
    nodes, years, window, channels = train_data.shape    #TAKE A LOOK HERE
#     print(train_data.shape)

    # train_data = train_data[:,:(num_years*2-1),:,:]
    # train_data = train_data[:,years-num_years*2+1:,:,:]
    nodes, years, window, channels = train_data.shape     #TAKE A LOOK HERE
#     print(train_data.shape)
    
    file, index = "strided_test", "out_indices"


    # BUILD MODEL
    inverse_model = getattr(MODEL, "ae")(input_channels=len(dynamic_channels)+len(output_channels), code_dim=latent_code_dim, hidden_dim=latent_code_dim, output_channels=len(static_channels), device=device)
    inverse_model = inverse_model.to(device)
    pytorch_total_params = sum(p.numel() for p in inverse_model.parameters() if p.requires_grad)
    print(inverse_model)
    forward_model = getattr(MODEL, "tamlstm")(input_dynamic_channels=len(dynamic_channels), input_static_channels=latent_code_dim, hidden_dim=forward_code_dim, output_channels=len(output_channels), dropout=dropout)
    forward_model = forward_model.to(device)
    pytorch_total_params += sum(p.numel() for p in forward_model.parameters() if p.requires_grad)
    print(forward_model)
    print("#Parameters:{}".format(pytorch_total_params))
    criterion = torch.nn.MSELoss(reduction="none")
    test_criterion = torch.nn.MSELoss(reduction="mean")
    optimizer_embedding = torch.optim.Adam(list(inverse_model.parameters()), lr=learning_rate)
    optimizer_forward = torch.optim.Adam(list(forward_model.parameters()), lr=learning_rate)
    # print("#Parameters:{}".format(pytorch_total_params))
    # print(model)

    # LOAD MODEL
    inverse_model.load_state_dict(torch.load(os.path.join(MODEL_DIR, "{}_inverse".format(model_name))))
    inverse_model.eval()
    forward_model.load_state_dict(torch.load(os.path.join(MODEL_DIR, "{}_forward".format(model_name))))
    dataset_code = unknown*np.ones((nodes, years, latent_code_dim), dtype=np.float32)


    for year in range(years):

        #Get instances for each node
        node_data = train_data[np.arange(nodes), year]
        # print(node_data.shape)

        # Remove pairs where node have unknown in streamflow
        keep_idx = np.zeros((node_data.shape[0], 1)).astype(bool)
        keep_idx[:,0] = (node_data[:,:,-1]!=unknown).all(axis=1)
        keep_idx = keep_idx.all(axis=1)
        node_data = node_data[keep_idx]
        # print(node_data.shape)

        for batch in range(math.ceil(node_data.shape[0]/batch_size)):

            # GET BATCH DATA FOR INVERSE MODEL
            batch_data = torch.from_numpy(node_data[batch*batch_size:(batch+1)*batch_size]).to(device)
            batch_input = batch_data[:,:,dynamic_channels+output_channels]
            batch_static = batch_data[:,0,static_channels]
            # print(batch_input.shape, batch_static.shape)

            # GET INVERSE OUTPUT
            batch_code_vec, batch_static_pred, batch_input_pred,_ = inverse_model(x=batch_input.float())
            # print(batch_code_vec.shape, batch_static_pred.shape, batch_input_pred.shape)

            # STORE OUTPUT
            dataset_code[batch*batch_size:(batch+1)*batch_size, year] = batch_code_vec.detach().cpu().numpy()
    dataset_code = np.mean(dataset_code, axis=1)
    print("dataset_code",dataset_code.shape)
    
    
    dataset_code = np.repeat(dataset_code, data_test.shape[0], axis=0)
    
    np.save(os.path.join(RESULT_DIR, "{}_{}_{}_{}_Embedding".format(file, index, few_shot_years, model_name)), dataset_code)


    end = time.time()
    print("Time:{:.4f}".format(end-start))
    

train data shape  (15, 1, 30, 24)
ae(
  (instance_encoder): Sequential(
    (0): Linear(in_features=6, out_features=64, bias=True)
    (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.2)
  )
  (temporal_encoder): LSTM(64, 64, batch_first=True, bidirectional=True)
  (code_linear): Linear(in_features=64, out_features=64, bias=True)
  (decode_linear): Linear(in_features=64, out_features=64, bias=True)
  (temporal_decoder): LSTM(64, 64, batch_first=True)
  (instance_decoder): Linear(in_features=64, out_features=6, bias=True)
  (static_out): Linear(in_features=64, out_features=5, bias=True)
)
tamlstm(
  (encoder): TAMLSTM(
    (_embeddings): ModuleList(
      (0): Linear(in_features=64, out_features=700, bias=True)
      (1): Linear(in_features=64, out_features=700, bias=True)
      (2): Linear(in_features=64, out_features=700, bias=True)
      (3): Linear(in_features=64, out_features=700, bias=True)
    )
  )
  (out): 